In [28]:
import pandas as pd
import tensorflow as tf 
import re 
from tensorflow.keras.layers import LSTM , Bidirectional , Embedding, Dense ,Dropout
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import TextVectorization
from collections import Counter
import gradio 

In [29]:
df = pd.read_csv('/kaggle/working/train.csv')

In [30]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [31]:
X = df['comment_text']
y = df.drop(columns = ['comment_text' , 'id'], axis = 1)

In [32]:
y = y.values 

In [33]:
MAX_WORDS = 200000  #number of words in vocab
vectorizer = TextVectorization(max_tokens=MAX_WORDS , #number of words in the vocab
    output_sequence_length = 1800 , #it the length of the outsequense  
    output_mode = 'int'#the word number is integear
    )

In [34]:
vectorizer.adapt(X.values)

In [35]:
model = Sequential([
    Embedding(MAX_WORDS +1 , 32),
    Bidirectional(LSTM(32, activation='tanh')),
   Dense(128, activation='relu'),
Dense(256, activation='relu'),
Dense(128, activation='relu'),
    Dense(6, activation='sigmoid')
])

In [36]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam',metrics=["accuracy"])

In [37]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [38]:
vectorized_text = vectorizer(X)
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
#cache the dataset 
dataset = dataset.cache()
#Shuffle the dataset 
dataset = dataset.shuffle(160000)
#batch the dataset
dataset = dataset.batch(16)

In [39]:
train_size = int(len(dataset) * 0.6)  # 60% for training
val_size = int(len(dataset) * 0.2)    # 20% for validation
test_size = len(dataset) - train_size - val_size  # Remaining for testing

train = dataset.take(train_size)

remaining_dataset = dataset.skip(train_size)
val = remaining_dataset.take(val_size)

remaining_dataset = remaining_dataset.skip(val_size)
test = remaining_dataset.take(test_size)

In [ ]:
history = model.fit(train, epochs=1, validation_data=val)

In [41]:
import pickle 

def save_tokenizer(tokenizer , filename):
    with open(filename  , 'wb') as f:
        pickle.dump(tokenizer, f)

    print("Tokenizer is Saved")

def load_tokenizer(filename):
    with open(filename, 'rb') as f:
        tokenizer = pickle.load(f)
    print(f"Tokenizer loaded from {filename}")
    return tokenizer
save_tokenizer(vectorizer ,'tokenizer.pkl' )
loaded_tokenizer = load_tokenizer('tokenizer.pkl')


Tokenizer is Saved
Tokenizer loaded from tokenizer.pkl


In [42]:
model

<Sequential name=sequential_2, built=True>

In [47]:
import gradio as gr

In [48]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [ ]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [ ]:
interface.launch(share=True)